In [6]:
import os, glob, corner, sys
import numpy as np
import pickle as pkl
import astropy.coordinates
from astropy.io import fits
import webbpsf
import subprocess
from galight.data_process_Danial import DataProcess
from galight.fitting_specify_Danial import FittingSpecify
from galight.fitting_process_Danial import FittingProcess

def galight_free_mod(object, filter, save_dir, nsigma, npixels, fit_bulge=True):

    # config
    program_ID = object.split('_')[0]
    object_ID = object.split('_')[1]
    save_path = f'{save_dir}/{program_ID}/{object_ID}'

    pixelscale = 0.04

    # read the data

    # reading the fits_files
    psf_file = f'{save_path}/{filter}_{pixelscale}.fits'
    PSF_data = fits.open(psf_file)[0].data
    
    sci_file = glob.glob(f'{save_path}/*{object_ID}*sci*')[0]
    sci_data = fits.open(sci_file)[0].data

    wht_file = glob.glob(f'{save_path}/*{object_ID}*wht*')[0]
    wht_data = fits.open(wht_file)[0].data
    err_data = np.power(wht_data, -0.5)

    # calculate the photometric zero point
    sci_header = fits.open(sci_file)[0].header
    zeropoint = 28.9
    # zeropoint = -2.5 * np.log10(sci_header['PHOTFLAM']) - 5 * np.log10(sci_header['PHOTPLAM']) - 2.408
    

    # the main target is at the centre of the FOV
    x_target, y_target = int(sci_data.shape[0]/2), int(sci_data.shape[1]/2)

    # run GALIGHT

    # GALIGHT pre-processing step
    data_process = DataProcess(fov_image=sci_data, fov_noise_map=err_data, header=sci_header,
                               target_pos=[x_target, y_target], pos_type='pixel',
                               rm_bkglight=True, if_plot=False, zp=zeropoint)

    radius = 1.2/pixelscale # arcsec/arcsec
    data_process.generate_target_materials(radius=radius, create_mask=False, if_plot=False,
                                           nsigma=nsigma, exp_sz=1.5, npixels=npixels,
                                           detect=True, detection_path=save_path)

    # add the PSF
    data_process.PSF_list = [PSF_data]

    # check if everything is there before attempting the run
    data_process.checkout()

    # fit bulge separately if specified with fit_bulge
    if fit_bulge == True:
        # modify the fitting of the component (i.e., galaxy) id = 0 into to components (i.e., bulge + disk)
        import copy
        apertures = copy.deepcopy(data_process.apertures)
        comp_id = 0 
        add_aperture0 = copy.deepcopy(apertures[comp_id])
        # this setting assigns comp0 as 'bulge' and comp1 as 'disk'
        add_aperture0.a, add_aperture0.b = add_aperture0.a/2, add_aperture0.b/2
        apertures = apertures[:comp_id] + [add_aperture0] + apertures[comp_id:]
        data_process.apertures = apertures # pass apertures to the data_process

        # adding a prior so that 1)the size of the bulge is within a range to the disk size, 2) disk have more ellipticity
        import lenstronomy.Util.param_util as param_util
        def condition_bulgedisk(kwargs_lens, kwargs_source, kwargs_lens_light, kwargs_ps, kwargs_special, kwargs_extinction, kwargs_tracer_source):
            logL = 0
            # note that the Comp[0] is the bulge and the Comp[1] is the disk
            phi0, q0 = param_util.ellipticity2phi_q(kwargs_lens_light[0]['e1'], kwargs_lens_light[0]['e2'])
            phi1, q1 = param_util.ellipticity2phi_q(kwargs_lens_light[1]['e1'], kwargs_lens_light[1]['e2'])
            cond_0 = (kwargs_lens_light[0]['R_sersic'] > kwargs_lens_light[1]['R_sersic'] * 0.9)
            cond_1 = (kwargs_lens_light[0]['R_sersic'] < kwargs_lens_light[1]['R_sersic']*0.15)
            cond_2 = (q0 < q1)
            if cond_0 or cond_1 or cond_2:
                logL -= 10**15
            return logL

    # set up the model
    fit_sepc = FittingSpecify(data_process)
    # the 'fix_n_list' will fix Sersic_n as 4 for the comp0 (bulge), and as 1 for the comp1 (disk).
    fit_sepc.prepare_fitting_seq(point_source_num = 0, fix_n_list= [[0,4], [1,1]]) 
    fit_sepc.build_fitting_seq()
    # fit_sepc.plot_fitting_sets()

    # set up the fitting method and run
    savename = f'{save_path}/galight_{filter}'
    fit_run = FittingProcess(fit_sepc, savename=savename, fitting_level=['shallow', 'deep'])
    fit_run.run(algorithm_list = ['MCMC', 'MCMC'])



    data = fit_run.fitting_specify_class.kwargs_data['image_data']
    noise = fit_run.fitting_specify_class.kwargs_data['noise_map']
    galaxy_list = fit_run.image_host_list
    galaxy_image = np.zeros_like(galaxy_list[0])
    for i in range(len(galaxy_list)):
        galaxy_image = galaxy_image+galaxy_list[i]
    model = galaxy_image
    norm_residual = (data - model)/noise

    # (data - central model) / central model
    # TODO: only use central profile
    norm_residual *= model # scale by sersic profile 
    return norm_residual


object = '2750_9973'
save_dir = '/Users/jmark/OneDrive/Desktop/DARK REU/morphology_fitting/saves'
nsigma = 1.9
npixels = 4

norm_residual = galight_free_mod(object, 'SW', save_dir, nsigma, npixels)

residual_sum = sum(map(sum, np.abs(norm_residual))) # sum absolute value of all elements of 2d array 
print('Residual sum: ', residual_sum)

Estimating the background light ... ... ...
The negative PSF values are corrected as 0 values.
The data_process is ready to go to pass to FittingSpecify!
The settings for the fitting is done. Ready to pass to FittingProcess. 
  However, please make updates manullay if needed.
MCMC selected. Sampling with default option emcee.


100%|██████████| 130/130 [00:27<00:00,  4.69it/s]


Computing the MCMC...
Number of walkers =  100
Burn-in iterations:  100
Sampling iterations (in current run): 130
27.909294843673706 time taken for MCMC sampling
MCMC selected. Sampling with default option emcee.
re-using previous samples to initialize the next MCMC run.


100%|██████████| 300/300 [00:51<00:00,  5.84it/s]


Computing the MCMC...
Number of walkers =  100
Burn-in iterations:  100
Sampling iterations (in current run): 300
51.58700156211853 time taken for MCMC sampling
79.567 total time taken for the overall fitting (s)
============ CONGRATULATION, YOUR JOB WAS SUCCESSFUL ================ 
Start transfering the Params to fluxs...
10000 MCMC samplers in total, finished translate: 0
10000 MCMC samplers in total, finished translate: 1000
10000 MCMC samplers in total, finished translate: 2000
10000 MCMC samplers in total, finished translate: 3000
10000 MCMC samplers in total, finished translate: 4000
10000 MCMC samplers in total, finished translate: 5000
10000 MCMC samplers in total, finished translate: 6000
10000 MCMC samplers in total, finished translate: 7000
10000 MCMC samplers in total, finished translate: 8000
10000 MCMC samplers in total, finished translate: 9000
Residual sum:  1.5279896201753143
